In [1]:
import glob
import pandas as pd

from os.path import basename
import pickle

from level2.level2_utils import *
from level2.level2_roi_extraction import level2_roi_extractor
from level2.level2_roi_extraction import load_rois, get_roi_data_for_beta
from level2.level2_roi_extraction import get_roi_data_for_l2_betas, get_roi_data_for_multirun_l2_betas
#import modules from files in a parallel directory "direct_regression"


from direct_regression.get_all_series import get_beta_img, get_roi_data, get_moment_trial_type_revealed, get_behavioral_data_with_moment_trial_type_revealed, mask_3d_subject_image
from direct_regression.get_all_series import get_all_subj_df
from direct_regression.fmri_utils import *



In [2]:

config = load_config("direct_regression/config.yml")
analysis_name = 'conditions'


#get the masks
mask_locations = [
    config['mask_location'] + 'aim3/masks/neurosynth/',
    config['mask_location'] + 'aim3/masks/'
]

signature_locations = [
    config['mask_location'] + 'signatures/aim3/'
]

roi_df = get_mask_df_from_mask_locations(mask_locations)
signature_df = get_mask_df_from_mask_locations(signature_locations)
roi_df['image_type'] = 'mask'
signature_df['image_type'] = 'signature'

#combine the two dfs
roi_df = pd.concat([roi_df, signature_df])
#get the list of raw nii files
glob_path = config['fmriprep_dir'] + config['nii_raw_path']

Benjamins-MacBook-Pro-2.local
{'dev_scripts_path': '/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/', 'level2_output_path': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/SST/', 'rural_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'rubric_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/DEV_scoring_rubrics', 'l2_analysis_files': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/', 'input_mat_path': '/Users/benjaminsmith/oregon/code/DEV_scripts/fMRI/fx/multiconds/SST/full_duration/conditions', 'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'dev_analysis_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/ana

## SST

### W1 W2 analysis

In [130]:
#get the masks
mask_locations = [
    config['mask_location'] + 'aim3/masks/neurosynth/',
    config['mask_location'] + 'aim3/masks/',
    config['mask_location'] + 'striatum/',
    config['mask_location']
]
wave_compare_roi_df = get_mask_df_from_mask_locations(mask_locations)

In [131]:
#filter the mask_label in mask_df, using regex, to only use stiraum, finger movements, motor control, and response inbhitioin
sst_roi_df = wave_compare_roi_df.loc[wave_compare_roi_df['mask_label'].str.contains('response inhibition|striatum_joint_reward|value')]


In [132]:

## SST
train_betas_with_data = get_data_for_confirmed_train_subjs(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    exclude_test_subjs=False
)[0:5]

train_betas_with_data['wave']='all'

loaded 275 rows from data_by_ppt.csv
255 subjects with no reported problems in task Redcap scanner notes, added to the provision useable_dev_id list.
7 subjects excludeable for missing scan data.
251 subjects remaining on the provision useable_dev_id list from the redcap list after excluding subjects with missing scan data.
217 subjects remaining on the provisional useable_dev_id list from the redcap list after excluding subjects excluded by motion quality process.
beta paths before exclusion: 250
beta paths after exclusion via nsc_subject_exclusions and the provision useable_dev_id list: 185


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useable_betas_with_data.sort_values('subject_id', inplace=True)
/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_9896/960989048.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_betas_with_data['wave']='all'


In [6]:
contrasts_with_paths = get_contrasts_for_betas(train_betas_with_data)

/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betas['spm_l2_targeted_contrast_filepath'] = None


/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV008/
0: Healthy_Go spmT_0001.nii
1: Healthy_NoGo spmT_0002.nii
2: Unhealthy_Go spmT_0003.nii
3: Unhealthy_NoGo spmT_0004.nii
4: Null_Cue spmT_0005.nii
5: Stop(Healthy>Unhealthy) spmT_0006.nii
6: Stop(Unhealthy>Healthy) spmT_0007.nii
7: Go(Healthy>Unhealthy) spmT_0008.nii
8: Go(Unhealthy>Healthy) spmT_0009.nii
9: Healthy_Go(W2-W1) spmT_0010.nii
10: Healthy_Go(W1-W2) spmT_0011.nii
11: Healthy_NoGo(W2-W1) spmT_0012.nii
12: Healthy_NoGo(W1-W2) spmT_0013.nii
13: Unhealthy_Go(W2-W1) spmT_0014.nii
14: Unhealthy_Go(W1-W2) spmT_0015.nii
15: Unhealthy_NoGo(W2-W1) spmT_0016.nii
16: Unhealthy_NoGo(W1-W2) spmT_0017.nii
17: Null_Cue(W2-W1) spmT_0018.nii
18: Null_Cue(W1-W2) spmT_0019.nii
19: Stop(Healthy>Unhealthy)(W2-W1) spmT_0020.nii
20: Stop(Healthy>Unhealthy)(W1-W2) spmT_0021.nii
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_cond

### spatial normalization

In [135]:
#from level2.level2_roi_extraction import level2_roi_extractor

contrast_name_list = [
    'Unhealthy_NoGo(W2-W1)',
    'Unhealthy_Go(W2-W1)',
    'Stop(Healthy>Unhealthy)(W2-W1)'
    ]

l2_roi_extractor = level2_roi_extractor(center_data=True,scale_data=True)
l2_roi_extractor.load_all_images_simultaneously=True
l2_roi_extractor.image_standardize=True
l2_roi_extractor.brain_mask_path = '/Users/benjaminsmith/anaconda/lib/python3.6/site-packages/nltools/resources/MNI152_T1_2mm_brain_mask.nii.gz'

#get the ROI data
roi_data_sst_conditions = l2_roi_extractor.get_roi_data_for_l2_contrasts(contrasts_with_paths, contrast_name_list, sst_roi_df)

response inhibition_association-test_z_FDR_0.01
producing the vector for this mask/series...
value_association-test_z_FDR_0.01
producing the vector for this mask/series...
striatum_joint_reward_mask
producing the vector for this mask/series...
value_association_frontal_medial_cortex
producing the vector for this mask/series...
Unhealthy_NoGo(W2-W1)
loading 5images...
...loaded.
['DEV008', 'DEV013', 'DEV014', 'DEV015', 'DEV016'], ['all', 'all', 'all', 'all', 'all'], Unhealthy_NoGo(W2-W1), response inhibition_association-test_z_FDR_0.01


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


proportion of image active: 0.01383226975097848
activity scalar is [ 0.23372159  0.26805397  0.22419931 -0.32014032  0.21864516]
['DEV008', 'DEV013', 'DEV014', 'DEV015', 'DEV016'], ['all', 'all', 'all', 'all', 'all'], Unhealthy_NoGo(W2-W1), value_association-test_z_FDR_0.01


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:545: UserWarning: Casting data from int32 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:274: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("

proportion of image active: 0.016650108360635286
activity scalar is [-0.3371406   0.45255724 -0.00754983 -0.30309204  0.18685578]
['DEV008', 'DEV013', 'DEV014', 'DEV015', 'DEV016'], ['all', 'all', 'all', 'all', 'all'], Unhealthy_NoGo(W2-W1), striatum_joint_reward_mask


...

proportion of image active: 0.016650108360635286
activity scalar is [ 0.22650371 -0.38712601 -0.44804054 -0.02345837  0.06649961]
['DEV008', 'DEV013', 'DEV014', 'DEV015', 'DEV016'], ['all', 'all', 'all', 'all', 'all'], Stop(Healthy>Unhealthy)(W2-W1), striatum_joint_reward_mask


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:545: UserWarning: Casting data from int32 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:274: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  ret

proportion of image active: 0.005345483279191523
activity scalar is [ 0.26390682 -0.37288797 -0.12177748 -0.12070618  0.16758881]
['DEV008', 'DEV013', 'DEV014', 'DEV015', 'DEV016'], ['all', 'all', 'all', 'all', 'all'], Stop(Healthy>Unhealthy)(W2-W1), value_association_frontal_medial_cortex


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:545: UserWarning: Casting data from int32 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:274: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  ret

proportion of image active: 0.0007864810288022106
activity scalar is [ 1.02881767  0.04526708 -1.61032048  0.62118094  0.23062074]


In [109]:
col_function=lambda img_name: "contrast_" + img_name + "_fname"
#iterate through each mask
#return(self.get_roi_data_for_l2(beta_list,condition_list,roi_df,col_function)
raw_roi_list = l2_roi_extractor.load_rois(roi_df)

#now iterate through each beta image
#for each beta type
# roi_data_all = []

print(condition)
    
#roi_data_sst_conditions = l2_roi_extractor.get_roi_data_for_l2_contrasts(contrasts_with_paths, contrast_name_list, sst_roi_df)
beta_list = contrasts_with_paths
condition_list = contrast_name_list
roi_df = sst_roi_df
#def get_roi_data_for_l2_contrasts(self, beta_list, condition_list,roi_df):
#return(self.get_roi_data_for_l2(beta_list,condition_list,roi_df,col_function))
#def get_roi_data_for_l2(self, beta_list,condition_list,roi_df,col_function):
#def get_roi_data_across_all_betas(self,beta_list: pd.DataFrame, condition,col_function, raw_roi_list, roi_df):
    # if self.load_all_images_simultaneously:
#roi_data = l2_roi_extractor.get_roi_data_across_all_betas(beta_list, condition,col_function, raw_roi_list, roi_df)
    # else:
    #     roi_data = self.get_roi_data_for_roi_col(beta_list, condition,col_function, raw_roi_list)
    # roi_data_all = roi_data_all + roi_data
    
#def get_roi_data_across_all_betas(self,beta_list: pd.DataFrame, condition,col_function, raw_roi_list, roi_df):
colname = col_function(condition)
image_series,beta_list_exists = l2_roi_extractor.get_image_series(beta_list,colname)



response inhibition_association-test_z_FDR_0.01
producing the vector for this mask/series...
value_association-test_z_FDR_0.01
producing the vector for this mask/series...
striatum_joint_reward_mask
producing the vector for this mask/series...
value_association_frontal_medial_cortex
producing the vector for this mask/series...
Unhealthy_NoGo(W2-W1)
loading 5images...
...loaded.


In [114]:
print(np.sum(np.sum(np.abs(image_fdata),axis=3)>0.1))
print(np.sum(np.sum(np.abs(image_fdata),axis=3)>0.01))
print(np.sum(np.sum(np.abs(image_fdata),axis=3)>0.001))
print(np.sum(np.sum(np.abs(image_fdata),axis=3)>0.0001))
print(np.sum(np.sum(np.abs(image_fdata),axis=3)>0.00001))
print(np.sum(np.sum(np.abs(image_fdata),axis=3)>0.000001))
print(np.sum(np.sum(np.abs(image_fdata),axis=3)>0.0000001))

228352
228352
228352
228352
228352
228352
228352


In [118]:
#put that back into an image
image_series_normed = nil.image.new_img_like(image_series,image_series_normed_fdata)

image_series.to_filename("test_image_raw.nii.gz")
image_series_normed.to_filename("test_image_normed.nii.gz")

I think we have work to do, see:

https://docs.google.com/spreadsheets/d/15iykB7qzCm8xqt7hgzIH58e-pAWgGz9BWKjEeN0MpBw/edit#gid=0


# TEMP

In [134]:
import os
import nilearn as nil
import nibabel as nib
import pandas as pd
import numpy as np
from direct_regression.get_all_series import get_beta_img, mask_3d_subject_image, mask_4d_subject_image, signature_weight_3d_subject_image

#for backwards compatibility, we expose some functions that work without the class

def get_roi_data_for_beta(
            subject_id,
            wave,
            spm_l2_path,
            condition,
            beta_name,
            raw_roi_list, roi_df, active_img_cleaned):
    return level2_roi_extractor().get_roi_data_for_beta(
            subject_id,
            wave,
            spm_l2_path,
            condition,
            beta_name,
            raw_roi_list, roi_df, active_img_cleaned)

def get_roi_data_for_l2_betas():
    return level2_roi_extractor().get_roi_data_for_l2_betas()

def load_rois(mask_df):
    return level2_roi_extractor().load_rois(mask_df)

def get_roi_data_for_multirun_l2_betas(beta_list,condition_list,mask_df,spatial_mean_center=False,
                                        collapse_runs=False):
    return level2_roi_extractor().get_roi_data_for_multirun_l2_betas(beta_list,condition_list,mask_df,spatial_mean_center=False,
                                        collapse_runs=False)

class level2_roi_extractor:

    # define init
    def __init__(self,center_data=False, scale_data=False):
        self.center_data = center_data
        self.scale_data = scale_data
        self.load_all_images_simultaneously = False
        self.image_standardize = False
        self.brain_mask_path = None


    def load_rois(self, mask_df):
        raw_mask_list = []
        for m_i, m_set in mask_df.iterrows():
            print(m_set['mask_label'])
            print('producing the vector for this mask/series...')
            #get the mask
            mask_raw = nil.image.load_img(m_set['mask_path'])
            raw_mask_list = raw_mask_list + [mask_raw]

        return raw_mask_list
    
    
    def spatially_normalize(self, image_series: nib.nifti1.Nifti1Image) -> nib.nifti1.Nifti1Image:
        """
        takes a 4d image series with 0s representing voxels outside the brain and every other value representing inside the brain
        creates an auto-mask from that image series
        then spatially normalizes the image series by subtracting the mean and dividing by the standard deviation
        """
        assert (len(image_series.shape)==4)
        #get a mean of the first three dimensions of the image
        #actually have to mask these first.

        image_fdata = image_series.get_fdata()
        image_defacto_mask = np.mean(np.abs(image_fdata),axis=3)>0.00001
        image_means = np.mean(image_fdata[image_defacto_mask],axis=0)
        image_sds = np.std(image_fdata[image_defacto_mask],axis=0)
        # print(image_means, image_sds)
        # print(np.mean(image_means), np.mean(image_sds))

        #now divide each voxel by the mean applicalbe to that voxel
        image_series_normed_fdata = image_fdata.copy()
        image_series_normed_fdata[image_defacto_mask]= (image_fdata[image_defacto_mask] - image_means)/image_sds

        image_series_normed = nil.image.new_img_like(image_series,image_series_normed_fdata)

        return(image_series_normed)


    
    def mask_subject_image(self,roi_raw, active_img_cleaned, bin_threshold):
        if len(active_img_cleaned.shape)==3 or active_img_cleaned.shape[3]==1:
            masked_img = mask_3d_subject_image(roi_raw, active_img_cleaned, bin_threshold)
        elif len(active_img_cleaned.shape)==4:
            masked_img = mask_4d_subject_image(roi_raw, active_img_cleaned, bin_threshold)
        return(masked_img)
    

    def get_roi_data_for_beta(self, 
            subject_id,
            wave,
            spm_l2_path,
            condition,
            beta_name,
            raw_roi_list, roi_df, active_img_cleaned):
        roi_data_for_beta = []
        roi_df = roi_df.copy().reset_index(drop=True)
        #rules for input: 
        # length of active_img_cleaned can be 1 or more
        # subject_id should be a scalar, a string, or an iterable with the same number of images in active_img_cleaned
        assert (len(subject_id)==active_img_cleaned.shape[3] or isinstance(subject_id,str) or (not hasattr(subject_id, "__iter__")))
        # length of beta_name should also be 1 or the same as the number of images in active_img_cleaned
        assert (len(beta_name)==active_img_cleaned.shape[3] or isinstance(beta_name,str) or (not hasattr(beta_name, "__iter__")))
        #if subject_id or wave are iterables, then beta_name should be, too
        if (
            (hasattr(subject_id, "__iter__") and subject_id is not str) or
            (hasattr(wave, "__iter__") and wave is not str)):
            assert (hasattr(beta_name, "__iter__") and beta_name is not str)
        # masks are always independent of the number of images in active_img_cleaned
        
        for m_i, m_set in roi_df.iterrows():
            print(str(subject_id) + ", " + str(wave) + ", " + str(condition) + ", " + m_set['mask_label'])
            
            roi_raw = raw_roi_list[m_i]

            if 'image_type' not in m_set.keys():
                #no image type set. need to handle this case first in order to avoid errors.
                active_img_masked = self.mask_subject_image(roi_raw, active_img_cleaned, bin_threshold=np.max(roi_raw.get_fdata())/1000)
            elif m_set['image_type'] == 'mask':
                #binary threshold has to be not quite zero because, with the dtype transform, 
                # some zeros get rounded up to a very-close-to-zero amount.
                active_img_masked = self.mask_subject_image(roi_raw, active_img_cleaned, bin_threshold=np.max(roi_raw.get_fdata())/1000)
            elif m_set['image_type'] == 'signature':
                #handles a bit differently; for signatures we don't set a binary threshold; the images are simply multiplied together.
                active_img_masked = signature_weight_3d_subject_image(roi_raw, active_img_cleaned)
            else:
                raise ValueError("image type not recognized")
            
            if len(active_img_masked.shape)==1:
                activity_scalar = active_img_masked.mean()
            elif len(active_img_masked.shape)==2:
                #get row means of the masked image
                activity_scalar = active_img_masked.mean(axis=1)


            print("activity scalar is " + str(activity_scalar))
            roi_data_for_beta.append(pd.DataFrame({
                'subject_id': subject_id,#r['subject_id'],
                'wave':wave,# r['wave'],
                'spm_l2_path':spm_l2_path,#r['spm_l2_path'],
                'condition' : condition,
                'beta_name': beta_name,#r[colname],
                'mask_label': m_set['mask_label'],
                'roi_activity': activity_scalar
            }))
        return roi_data_for_beta



    def get_roi_data_for_multirun_l2_betas(self, beta_list,condition_list,mask_df,spatial_mean_center=False,
                                        collapse_runs=False):
        ###
        #takes a beta list where each entry is one beta image 
        #mean-centers each mask
        #and gets the ROI data for each beta image
        
        
        raw_mask_list = load_rois(mask_df)

        roi_data_all = []

        #iterate through each subject and wave
        for sub_i,sub in enumerate(np.unique(beta_list.subject_id)):
            #and through each condition
            for wave_i,wave in enumerate(np.unique(beta_list.wave)):
                #get the conditions applicable for this subject
                session_betas = beta_list[(beta_list['subject_id']==sub) & (beta_list['wave']==wave)]
                #iterate through each condition
                for condition in set(np.unique(session_betas.beta_description)).intersection(set(condition_list)):
                    #get the betas for this condition
                    condition_betas = session_betas[session_betas['beta_description']==condition]


                    #iterate through each beta
                    for i, r in condition_betas.iterrows():
                        #some checks where skipping is necessary
                        if type(r['beta_fname'])==float:
                            if np.isnan(r['beta_fname']):
                                continue
                            #we want an error thrown if somehow it's a float but not nan,
                            # so we handle that case here.
                        if r['beta_fname'] is None:
                            continue
                        
                    
                        #get the beta image from which to pull an ROI
                        beta_img_filepath = r['spm_l2_path'] + r['beta_fname']
                        active_img_cleaned = get_beta_img(beta_img_filepath)
                        if active_img_cleaned is None:
                            continue

                        print("img mean is " + str(np.mean(active_img_cleaned.get_fdata()[np.isnan(active_img_cleaned.get_fdata())==False])))


                        if spatial_mean_center:
                            #mean center the nifti image
                            raise NotImplementedError("spatial mean centering not implemented")
                            active_img_cleaned = active_img_cleaned - active_img_cleaned.mean()


                            #active_img_cleaned = active_img_cleaned - active_img_cleaned.mean()

                        #for each mask
                        roi_data_from_masks = get_roi_data_for_beta(
                                r['subject_id'],
                                r['wave'],
                                r['spm_l2_path'],
                                condition,
                                r['beta_fname'],
                                raw_mask_list, mask_df, active_img_cleaned)
                        roi_data_from_masks_df = pd.DataFrame(roi_data_from_masks)
                        roi_data_from_masks_df['run'] = r['task_run']
                        roi_data_all.append(roi_data_from_masks_df)
                        
        roi_data_all_df = pd.concat(roi_data_all)

        if collapse_runs:
            roi_data_all_df = collapse_runs(roi_data_all_df)

        return roi_data_all_df
    


    def get_roi_data_for_l2_contrasts(self, beta_list, condition_list,roi_df):
        col_function=lambda img_name: "contrast_" + img_name + "_fname"
        #iterate through each mask
        return(self.get_roi_data_for_l2(beta_list,condition_list,roi_df,col_function))

    def subtract_mean(self, image_series):
        img_mask = self.brain_mask_path
                # Get a mean image within the roi
        mean_image = nil.image.math_img("np.mean(img1 * img2, axis=-1)",
                                img1=image_series, img2=img_mask)
        
        # duplicate the mean image to have the same shape as image_series
        mean_image = nil.image.new_img_like(image_series, 
                                        np.repeat(mean_image.get_fdata()[..., np.newaxis], 
                                                image_series.shape[-1], axis=-1))

        # Subtract the mean from each image
        standardized_images = nil.image.math_img('(img1 - img2)', img1=image_series, img2=mean_image)
        return(standardized_images)
    
    def get_image_series(self, beta_list,colname):
            
        beta_filepaths = [r['spm_l2_path'] + r[colname] for i, r in beta_list.iterrows()]
        #strip out the filepaths that don't exist
        filepath_exists = [os.path.exists(b) for b in beta_filepaths]
        beta_list_exists = beta_list[filepath_exists].reset_index(drop=True,inplace=False)
        del beta_list

        #get just the beta_filepaths that exist by indexing beta_filepaths with the boolean array
        beta_filepath_exists = [beta_filepaths[i] for i in range(len(beta_filepaths)) if filepath_exists[i]]

        print("loading " + str(len(beta_filepath_exists)) + "images...")
        image_series = nil.image.load_img(beta_filepath_exists)
        print("...loaded.")
        return(image_series,beta_list_exists)


    def get_roi_data_across_all_betas(self,beta_list: pd.DataFrame, condition,col_function, raw_roi_list, roi_df):

        colname = col_function(condition)
        image_series,beta_list_exists = self.get_image_series(beta_list,colname)

        #image_series_cleaned = nil.image.clean_img(image_series,detrend=self.center_data ,standardize=self.scale_data)

        if self.image_standardize:
            #get the f
            image_series_cleaned = self.spatially_normalize(image_series)

        #image_series_cleaned.to_filename('cleaned_img.nii.gz')
        # img_cleaned_fdata = image_series_cleaned.get_fdata()
        # #get the mean over the image series
        # img_mean = nil.image.mean_img(image_series_cleaned)
        # #plot
        # nil.plotting.plot_img(img_mean)

        roi_data_for_beta = self.get_roi_data_for_beta(
                    beta_list_exists['subject_id'].tolist(),
                    beta_list_exists['wave'].tolist(),
                    beta_list_exists['spm_l2_path'].tolist(),
                    condition,
                    beta_list_exists[colname].tolist(),
                    raw_roi_list, roi_df, image_series_cleaned)
        
        roi_data_all = roi_data_for_beta

        return(roi_data_all)




    def get_roi_data_for_roi_col(self,beta_list, condition,col_function, raw_roi_list, roi_df):

        colname = col_function(condition)

        roi_data_all = []

        #iterate through each image, if it exists, and pull the ROI
        for i, r in beta_list.iterrows():
            #print(r['subject_id'] + ", " + str(r['wave']))xx
            #print(r.index)
            #some checks where skipping is necessary
            if colname not in beta_list.columns:
                continue
            if type(r[colname])==float:
                if np.isnan(r[colname]):
                    continue
                #we want an error thrown if somehow it's a float but not nan,
                # so we handle that case here.
            if r[colname] is None:
                continue
            
        
            #get the beta image from which to pull an ROI
            beta_img_filepath = r['spm_l2_path'] + r[colname]
            active_img_cleaned = get_beta_img(beta_img_filepath)

            if self.center_data or self.scale_data:
                raise NotImplementedError("centering and scaling not implemented")

            if active_img_cleaned is None:
                continue

            #for each mask
            roi_data_for_beta = self.get_roi_data_for_beta(
                    r['subject_id'],
                    r['wave'],
                    r['spm_l2_path'],
                    condition,
                    r[colname],
                    raw_roi_list, roi_df, active_img_cleaned)
            
            roi_data_all.append(pd.DataFrame(roi_data_for_beta))
            
        return(roi_data_all)
    
    def get_roi_data_for_l2(self, beta_list,condition_list,roi_df,col_function):
        raw_roi_list = self.load_rois(roi_df)
        #now iterate through each beta image
        #for each beta type
        roi_data_all = []
        for condition in condition_list:
            print(condition)
            
            if self.load_all_images_simultaneously:
                roi_data = self.get_roi_data_across_all_betas(beta_list, condition,col_function, raw_roi_list, roi_df)
            else:
                roi_data = self.get_roi_data_for_roi_col(beta_list, condition,col_function, raw_roi_list)
            roi_data_all = roi_data_all + roi_data
            

        roi_data_df = pd.concat(roi_data_all)
        return roi_data_df


    def get_roi_data_for_l2_betas(self, beta_list, condition_list,roi_df):
        col_function=lambda img_name: "beta_" + img_name + "_fname"
        #iterate through each mask
        return(self.get_roi_data_for_l2(beta_list,condition_list,roi_df,col_function))

